# GPU Access on Apple Silicone: MacBook Pro

Version 0.1 
Date: August 2, 2023    
Author: Daniel M. Ringel    
Contact: dmr@unc.edu

*Daniel M. Ringel, Creating Synthetic Experts with Generative Artificial Intelligence (July 15, 2023).  
Available at SSRN: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4542949*

> Notes: this worked on April 27, 2023 for me



# 1. Set-up PyTorch on Macbook Pro 2023 M2 Max

0. (if not already installed) Install Xcode Command Line Tool
> xcode-select --install
1. Download and install Homebrew from https://brew.sh. Follow the steps it prompts you to go through after installation.
2. Download Miniforge3 (Conda installer) for macOS arm64 chips (M1, M1 Pro, M1 Max, M1 Ultra, M2):   
- Download: https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh 
- See all available versions here: https://docs.conda.io/en/latest/miniconda.html 
3. Open Terminal and run these commands to install Miniforge3 into home directory.
> chmod +x ~/Downloads/Miniforge3-MacOSX-arm64.sh  
sh ~/Downloads/Miniforge3-MacOSX-arm64.sh  
source ~/miniforge3/bin/activate

4. Restart terminal

5. Create a directory to setup a PyTorch environment 
> cd Documents  
mkdir M2GPUENV  
cd M2GPUENV

6. Make and activate a Conda environment.
 - I used Python 3.8 for stability
 > conda create --prefix ./env python=3.8  
conda activate ./env

7. Install PyTorch default version for Mac with pip (see https://pytorch.org/get-started/locally/?ref=mrdbourke.com) 
> pip3 install torch torchvision torchaudio

8. Install common data science packages
> conda install jupyter pandas numpy matplotlib scikit-learn tqdm scipy

9. Start Jupyter (make sure you are in Documents/M2GPUENV and that you activated the environment (*conda activate ./env*)
> jupyter notebook

10. Create a new notebook with "New" -> "Notebook: Python 3 (ipykernel)" and run the following code to check that it worked

In [ ]:
import torch

# Set Device
print(f"PyTorch version: {torch.__version__}")
if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"
    print("MPS (Apple Metal Performance Shader) is available")
elif torch.cuda.is_available():
    device = "cuda"
    print("CUDA (GPU) is available")
else:
    device = "cpu"
    print("Neiter MPS nor GPU available")
print(f"Using device: {device}")

In [ ]:
# Create data and send it to the device
x = torch.rand(size=(3, 4)).to(device)
x

# 2. Set-up Tensorflow on Macbook Pro 2023 M2 Max

- see https://developer.apple.com/metal/tensorflow-plugin/ 

### 2.1 Prep: *Prerequisites (already done for PyTorch setup)*
0. (if not already installed) Install Xcode Command Line Tool
> xcode-select --install
1. (if not already installed) Download and install Homebrew from https://brew.sh. Follow the steps it prompts you to go through after installation.
2. (if not already installed) Download Miniforge3 (Conda installer) for macOS arm64 chips (M1, M1 Pro, M1 Max, M1 Ultra, M2):   
- Download: https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh 
- See all available versions here: https://docs.conda.io/en/latest/miniconda.html 
3. (if not already installed) Open Terminal and run these commands to install Miniforge3 into home directory.
> chmod +x ~/Downloads/Miniforge3-MacOSX-arm64.sh  
sh ~/Downloads/Miniforge3-MacOSX-arm64.sh  
source ~/miniforge3/bin/activate

4. Restart terminal
5. Create a directory to setup a PyTorch environment 
> cd Documents  
mkdir M2GPUENV  
cd M2GPUENV

6. Make and activate a Conda environment.
 - I used Python 3.8 for stability
 > conda create --prefix ./env python=3.8  
conda activate ./env

### 2.2 Start Here: *Will use same environment as for PyTorch*
1. Switch in working directory
> cd Documents/M2GPUENV

2. Activate Python Environment
> conda activate ./env

3. Install TensorFlow dependencies
> conda install -c apple tensorflow-deps 
- Alternatively: Do this globally in your miniconda3 folder instead of the specific environment

4. Install base TensorFlow:
> pip install tensorflow-macos

5. Install Metal plugin:
> pip install tensorflow-metal

6. Start Jupyter Notebook
> jupyter notebook

7. Create a new notebook with "New" -> "Notebook: Python 3 (ipykernel)" and run the following code to check that it worked


In [ ]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")


In [ ]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=False,
    weights=None,
    input_shape=(32, 32, 3)
)
x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
output = tf.keras.layers.Dense(100, activation='softmax')(x)
model = tf.keras.models.Model(inputs=model.input, outputs=output)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=64)